\
            # 54. 数据分析基础：NumPy（NumPy Fundamentals）

            目标：掌握 NumPy 的数组模型与向量化思维，能写出正确、可读、相对高性能的数值代码。
本章依赖 `numpy`；若未安装，会提示安装命令并跳过相关演示。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- Python 列表/切片
- 函数与异常
- 了解矩阵/向量概念更佳


## 知识点地图

- 1. 为什么需要 NumPy：ndarray + 向量化
- 2. 安装与导入（可选依赖）
- 3. 创建数组：array/arange/zeros/ones/linspace
- 4. shape/dtype：数组的“形状”和“元素类型”
- 5. 索引与切片：一维、多维、切片视图
- 6. 广播（broadcasting）：不同形状如何相加
- 7. 向量化：用数组表达式替代 for 循环
- 8. 布尔索引与 where：过滤与条件赋值
- 9. 聚合与轴（axis）：sum/mean 与按行按列统计
- 10. 线性代数入门：dot、矩阵乘法 @


## 自检清单（学完打勾）

- [ ] 理解 ndarray、shape、dtype 的含义
- [ ] 会创建数组（array/arange/zeros/ones/linspace）并做基本索引/切片
- [ ] 理解广播（broadcasting）与向量化（vectorization）
- [ ] 会用布尔索引与条件选择（mask/where）
- [ ] 理解视图 vs 拷贝（view vs copy）避免隐蔽 bug
- [ ] 会做基础统计与线性代数接口（sum/mean/dot）


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：为什么需要 NumPy：ndarray + 向量化

- Python list 是“对象数组”：每个元素是一个 Python 对象，循环开销大。
- NumPy ndarray 是“同质连续内存”：dtype 固定，批量运算在底层实现，通常更快。

思维转换：
- 少写 Python for 循环
- 多用“向量化表达”（一次对整个数组操作）


## 知识点 2：安装与导入（可选依赖）

安装：
- `pip install numpy`

本章代码块都会先尝试 import；若失败会提示，不会中断 notebook。


In [ ]:
try:
    import numpy as np
except Exception as e:
    np = None
    print('numpy not available:', type(e).__name__, e)
    print('install: pip install numpy')
else:
    print('numpy version:', np.__version__)


## 知识点 3：创建数组：array/arange/zeros/ones/linspace

常见创建方式：
- `np.array([...])`
- `np.arange(start, stop, step)`（类似 range，但生成数组）
- `np.zeros(shape)` / `np.ones(shape)`
- `np.linspace(a, b, n)`（等距取 n 个点）

推荐：用明确 dtype（例如 float32/float64）避免隐式转换。


In [ ]:
try:
    import numpy as np
except Exception as e:
    print('install numpy to run this cell')
else:
    a = np.array([1, 2, 3], dtype=np.int64)
    b = np.arange(0, 10, 2)
    c = np.zeros((2, 3), dtype=np.float32)
    d = np.linspace(0.0, 1.0, 5)
    print('a', a, a.dtype, a.shape)
    print('b', b)
    print('c\n', c)
    print('d', d)


## 知识点 4：shape/dtype：数组的“形状”和“元素类型”

- shape：多维数组每一维的长度，例如 (2,3) 表示 2 行 3 列。
- dtype：元素类型（int32/float64/...），决定内存布局与运算规则。

常见坑：
- 不同 dtype 运算会发生类型提升（int + float -> float）。
- 用 Python int 生成的数组，dtype 可能随平台变化（常见 int64）。


In [ ]:
try:
    import numpy as np
except Exception:
    print('install numpy to run this cell')
else:
    x = np.array([1, 2, 3])
    y = np.array([1.0, 2.0, 3.0])
    print('x dtype:', x.dtype)
    print('y dtype:', y.dtype)
    print('(x + y) dtype:', (x + y).dtype, x + y)


## 知识点 5：索引与切片：一维、多维、切片视图

- `arr[i]` / `arr[i:j:k]`：一维索引/切片
- 多维：`arr[r, c]`，切片如 `arr[:, 1:]`

重要：
- NumPy 的切片通常返回 **视图（view）**，修改会影响原数组。
- 用 `.copy()` 明确复制，避免“改了切片把原数组改了”。


In [ ]:
try:
    import numpy as np
except Exception:
    print('install numpy to run this cell')
else:
    m = np.arange(12).reshape(3, 4)
    s = m[:, 1:3]
    s[0, 0] = 999
    print('m\n', m)
    print('slice s\n', s)
    t = m[:, 1:3].copy()
    t[0, 0] = -1
    print('copy t\n', t)
    print('m after copy edit\n', m)


## 知识点 6：广播（broadcasting）：不同形状如何相加

广播规则（直觉版）：
- 从末尾维度对齐比较
- 维度相同或其中一个为 1 才能广播

用途：
- 给每行/每列加一个向量
- 快速构造网格/归一化


In [ ]:
try:
    import numpy as np
except Exception:
    print('install numpy to run this cell')
else:
    m = np.arange(12).reshape(3, 4)
    col = np.array([10, 20, 30]).reshape(3, 1)
    row = np.array([1, 2, 3, 4])
    print('m + row\n', m + row)
    print('m + col\n', m + col)


## 知识点 7：向量化：用数组表达式替代 for 循环

向量化通常更简洁、更快：
- `y = a * x + b`
- `np.sin(x)`、`np.exp(x)` 等 ufunc

注意：
- 不要为了“看起来向量化”写出难懂表达式；可读性优先。


In [ ]:
try:
    import numpy as np
except Exception:
    print('install numpy to run this cell')
else:
    x = np.linspace(0, 1, 6)
    y = 2 * x + 1
    z = np.sin(2 * np.pi * x)
    print('x', x)
    print('y', y)
    print('z', z)


## 知识点 8：布尔索引与 where：过滤与条件赋值

- mask：`x[x > 0]` 过滤
- 条件赋值：`x[x < 0] = 0`
- `np.where(cond, a, b)`：三元选择

常见用途：异常值裁剪、缺失值填充（配合 NaN）、标签生成。


In [ ]:
try:
    import numpy as np
except Exception:
    print('install numpy to run this cell')
else:
    x = np.array([-2, -1, 0, 1, 2], dtype=float)
    pos = x[x > 0]
    x2 = x.copy()
    x2[x2 < 0] = 0
    y = np.where(x > 0, 1, 0)
    print('pos', pos)
    print('clipped', x2)
    print('labels', y)


## 知识点 9：聚合与轴（axis）：sum/mean 与按行按列统计

- `axis=None`：整体
- `axis=0`：沿着行方向压缩（按列统计）
- `axis=1`：沿着列方向压缩（按行统计）

建议：多画一张 shape 变化图，避免 axis 搞反。


In [ ]:
try:
    import numpy as np
except Exception:
    print('install numpy to run this cell')
else:
    m = np.arange(12).reshape(3, 4)
    print('m\n', m)
    print('sum all', m.sum())
    print('sum axis=0 (per col)', m.sum(axis=0))
    print('sum axis=1 (per row)', m.sum(axis=1))


## 知识点 10：线性代数入门：dot、矩阵乘法 @

- 点积/矩阵乘法：
  - `np.dot(a, b)`
  - `a @ b`（更推荐，语义更清晰）

注意：
- `*` 是逐元素乘法，不是矩阵乘法。


In [ ]:
try:
    import numpy as np
except Exception:
    print('install numpy to run this cell')
else:
    a = np.array([[1, 2], [3, 4]])
    b = np.array([[10, 20], [30, 40]])
    print('a*b\n', a * b)
    print('a@b\n', a @ b)


## 常见坑

- 把切片当拷贝：其实是 view，修改会影响原数组
- 广播维度没对齐：shape 不匹配导致 ValueError
- 混淆 * 与 @：逐元素 vs 矩阵乘法
- dtype 不一致导致精度/溢出问题（例如 int 溢出）
- 为了向量化牺牲可读性：复杂表达式难维护


## 综合小案例：实现一个“标准化 + 线性回归预测（最小版）”

用 NumPy 完成：
- 构造一组二维特征 X 与目标 y
- 对 X 做标准化（按列减均值除标准差）
- 用最小二乘求解线性回归系数（正规方程）：
  `w = (X^T X)^{-1} X^T y`（教学用；真实工程更推荐稳定算法）


In [ ]:
try:
    import numpy as np
except Exception:
    print('install numpy to run this cell')
else:
    # toy data
    X = np.array([[1, 10], [2, 20], [3, 30], [4, 40]], dtype=float)
    y = np.array([3, 6, 9, 12], dtype=float)  # roughly y = 0.3*x2

    mu = X.mean(axis=0)
    sigma = X.std(axis=0)
    Xn = (X - mu) / sigma

    # add bias term
    Xb = np.concatenate([np.ones((Xn.shape[0], 1)), Xn], axis=1)
    w = np.linalg.inv(Xb.T @ Xb) @ (Xb.T @ y)

    pred = Xb @ w
    print('mu', mu)
    print('sigma', sigma)
    print('w', w)
    print('pred', pred)


## 自测题（不写代码也能回答）

- ndarray 的 shape 与 dtype 分别表示什么？
- 为什么 NumPy 切片通常是 view？这带来什么好处与风险？
- 广播的基本规则是什么？
- 为什么向量化通常比 Python for 快？
- axis=0 与 axis=1 分别对应“按列/按行”吗？你怎么记？


## 练习题（建议写代码）

- 写一个函数：对二维数组每列做 min-max 归一化。
- 实现一个“滑动窗口均值”计算（用切片/向量化尽量避免 for）。
- 用布尔索引实现 winsorize（把极端值裁剪到指定分位数，了解）。
